In [1]:
import os
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

from PIL import Image
import pathlib 
from sklearn.model_selection import train_test_split
import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
import os

lfw_root = 'C:/Users/Mahesh/OneDrive/Desktop/PyTorch/DATASETS/FaceForensics/real_images_lfw'

real_image_paths = []

for root, dirs, files in os.walk(lfw_root):
    for file in files:
        if file.endswith(".jpg"):
            image_path = os.path.join(root, file)
            real_image_paths.append(image_path)

print(f"Total real images: {len(real_image_paths)}")


Total real images: 13233


In [5]:
deepfake_root = 'C:/Users/Mahesh/OneDrive/Desktop/PyTorch/DATASETS/FaceForensics/deepfake_images'

deepfake_image_paths = []

for root, dirs, files in os.walk(deepfake_root):
    for file in files:
        if file.endswith(".png"): 
            image_path = os.path.join(root, file)
            deepfake_image_paths.append(image_path)

print(f"Total deepfake images: {len(deepfake_image_paths)}")


Total deepfake images: 20351


In [6]:
all_image_paths = real_image_paths + deepfake_image_paths

labels = [0] * len(real_image_paths) + [1] * len(deepfake_image_paths)

print(f"Total images: {len(all_image_paths)}")
print(f"Total labels: {len(labels)}")


Total images: 33584
Total labels: 33584


In [7]:
from sklearn.model_selection import train_test_split

train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(
    all_image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)


In [8]:
from torch.utils.data import Dataset

class DeepfakeDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert('RGB') 

        if self.transform:
            image = self.transform(image)

        return image, label


In [9]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((32, 32)), 
    transforms.ToTensor(),  
])

train_dataset = DeepfakeDataset(train_image_paths, train_labels, transform=transform)
test_dataset = DeepfakeDataset(test_image_paths, test_labels, transform=transform)

In [10]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
print(f"Training Size : {len(train_image_paths)} samples.")
print(f"Testing Size  : {len(test_image_paths)} samples.")

Training Size : 26867 samples.
Testing Size  : 6717 samples.


In [12]:
import torch
import timm
import torch.nn as nn

model = timm.create_model('vit_base_patch16_224', pretrained=True, img_size = 32)

num_features = model.head.in_features
model.head = nn.Linear(num_features, 1)


c:\Users\Mahesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCEWithLogitsLoss()


In [16]:
num_epochs = 10 
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(images).squeeze()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc*100:.2f}%")

    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()

            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            predicted = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(test_loader)
    epoch_acc = correct / total
    test_losses.append(epoch_loss)
    test_accuracies.append(epoch_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {epoch_loss:.4f}, Test Accuracy: {epoch_acc*100:.2f}%")

c:\Users\Mahesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  x = F.scaled_dot_product_attention(


Epoch [1/10], Train Loss: 0.0347, Train Accuracy: 98.62%
Epoch [1/10], Test Loss: 0.0167, Test Accuracy: 99.48%
Epoch [2/10], Train Loss: 0.0066, Train Accuracy: 99.83%
Epoch [2/10], Test Loss: 0.0065, Test Accuracy: 99.87%
Epoch [3/10], Train Loss: 0.0040, Train Accuracy: 99.90%
Epoch [3/10], Test Loss: 0.0009, Test Accuracy: 99.97%
Epoch [4/10], Train Loss: 0.0077, Train Accuracy: 99.79%
Epoch [4/10], Test Loss: 0.0004, Test Accuracy: 99.97%
Epoch [5/10], Train Loss: 0.0024, Train Accuracy: 99.94%
Epoch [5/10], Test Loss: 0.0098, Test Accuracy: 99.70%
Epoch [6/10], Train Loss: 0.0187, Train Accuracy: 99.55%
Epoch [6/10], Test Loss: 0.0018, Test Accuracy: 99.94%
Epoch [7/10], Train Loss: 0.0046, Train Accuracy: 99.90%
Epoch [7/10], Test Loss: 0.0047, Test Accuracy: 99.93%
Epoch [8/10], Train Loss: 0.0043, Train Accuracy: 99.89%
Epoch [8/10], Test Loss: 0.0053, Test Accuracy: 99.85%
Epoch [9/10], Train Loss: 0.0102, Train Accuracy: 99.68%
Epoch [9/10], Test Loss: 0.0306, Test Accuracy:

In [17]:
import torch
from torchvision import transforms
from PIL import Image
import gradio as gr


model.eval()

def predict(image):
    image = transform(image).unsqueeze(0).to(device)  
    with torch.no_grad():
        output = model(image)
        probability = torch.sigmoid(output).item()

    if probability > 0.5:
        return "Deepfake", probability
    else:
        return "Real", probability


In [19]:
import gradio as gr

interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Label(num_top_classes=1), gr.Number(label="Confidence Score")],
    title="Deepfake Recognition",
    description="Upload an image to check if it is real or deepfake."
)
interface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
